In [ ]:
import pandas as pd

In [ ]:
users = pd.read_csv('Users.csv')
books = pd.read_csv('Books.csv')
ratings = pd.read_csv('Ratings.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Users.csv'

In [ ]:
users.head(10)

In [ ]:
books.shape

In [ ]:
ratings.head(10)

In [ ]:
users.info()
print(users.shape)

In [ ]:
users.describe()

In [ ]:
books.info()
print(books.shape)

In [ ]:
books.describe()

In [ ]:
ratings.info()
print(ratings.shape)

In [ ]:
a = users['Age'].isna().sum()
b = len(users['Age'])
print(a/b)

almost 40% age data is missing

In [ ]:
books.isna().sum()

In [ ]:
users.isna().sum()

In [ ]:
ratings.isna().sum()

In [ ]:
print(users.duplicated().sum())
print(books.duplicated().sum())
print(ratings.duplicated().sum())

no duplicate values

In [ ]:
ratings_new = ratings.merge(books,on = 'ISBN')
ratings_new

In [ ]:
ratings_df = ratings_new.groupby('Book-Title').count()['Book-Rating'].reset_index()
ratings_df.rename(columns = {'Book-Rating':'num_ratings'},inplace = True)
ratings_df






In [ ]:
avg_ratings_df = ratings_new.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_ratings_df.rename(columns = {'Book-Rating':'avg_ratings'},inplace = True)
avg_ratings_df

In [ ]:
popularity_df = ratings_df.merge(avg_ratings_df,on = 'Book-Title')
popularity_df

In [ ]:
popularity_df = popularity_df[popularity_df['num_ratings']>100].sort_values('avg_ratings',ascending = False).head(50)
popularity_df

In [ ]:
popularity_df = popularity_df.merge(books,on = 'Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Year-Of-Publication','Publisher','num_ratings','avg_ratings','Image-URL-M']]

In [ ]:
popularity_df

# New Section
**collaborative filtering**

In [ ]:
x = ratings_new.groupby('User-ID').count()['Book-Rating']>200
rated_users = x[x].index
rated_users

In [ ]:
filter_ratings = ratings_new[ratings_new['User-ID'].isin(rated_users)]
filter_ratings

now we will filter the filter_rating df on books with rating higher than 50

In [ ]:
y = filter_ratings.groupby('Book-Title').count()['Book-Rating']>=40
famous_books = y[y].index

In [ ]:
famous_books

In [ ]:
final_ratings = filter_ratings[filter_ratings['Book-Title'].isin(famous_books)]

In [ ]:
final_ratings

In [ ]:
pt = final_ratings.pivot_table(index = 'Book-Title',columns = 'User-ID',values = 'Book-Rating')

In [ ]:
pt.fillna(0,inplace = True)

In [ ]:
pt

In [ ]:
import tensorflow as tf

In [ ]:
tf.compat.v1.enable_eager_execution()

In [ ]:





# Convert the pivot table to a TensorFlow tensor
#pivot_tensor = tf.convert_to_tensor(pt.values, dtype=tf.float32)

# Calculate cosine similarity using TensorFlow
#similarity = tf.keras.losses.cosine_similarity(pivot_tensor, pivot_tensor, axis=1)



# You can run a TensorFlow session to get the similarity matrix
with tf.compat.v1.Session() as sess:
    pivot_tensor = tf.convert_to_tensor(pt.values, dtype=tf.float32)
    similarity = tf.keras.losses.cosine_similarity(pivot_tensor,pivot_tensor)

    cosine_similarity_matrix = sess.run(similarity)

# The cosine_similarity_matrix contains the cosine similarities between rows (users or items) in your pivot table.

In [ ]:
cosine_similarity_matrix.shape


In [ ]:
pivot_tensor = tf.convert_to_tensor(pt.values, dtype=tf.float32)
norm = tf.norm(pivot_tensor,axis = 1,keepdims = True)
normalized_matrix = pivot_tensor/norm
cosine_sim_matrix = tf.matmul(normalized_matrix,normalized_matrix,transpose_b = True)


In [ ]:
cosine_sim_matrix

recommend on book name


In [ ]:
final_books = books.merge(ratings,on = 'ISBN').drop_duplicates('Book-Title')
final_books

In [ ]:
def recommend(book_name):
  index = np.where(pt.index == book_name)[0][0]
  similar_items = sorted(list(enumerate(cosine_sim_matrix[index])),key = lambda x:x[1],reverse = True)[0:50]
  data = []

  for i in similar_items:
    item = []
    temp_df = final_books[final_books['Book-Title']==pt.index[i[0]]]
    item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
    item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
    item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
    #item.extend(list(temp_df.drop_duplicates('Book-Title')['Year-Of-Publication'].values))
    item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Rating'].values))


    data.append(item)

    #print(pt.index[i[0]])
  return(data)

In [ ]:
import numpy as np

In [ ]:
np.where(pt.index=='Wuthering Heights')[0][0]

In [ ]:
recommend('Wuthering Heights')

In [ ]:
recommend('Message in a Bottle')

author based recommender system


In [ ]:
pt1 = final_ratings.pivot_table(index = 'Book-Author',columns = 'User-ID',values = 'Book-Rating')

In [ ]:
pt1

In [ ]:
pt1.fillna(0,inplace = True)

In [ ]:
pt1

In [ ]:
pivot_tensor_author = tf.convert_to_tensor(pt1.values, dtype=tf.float32)
norm = tf.norm(pivot_tensor_author,axis = 1,keepdims = True)
normalized_matrix = pivot_tensor_author/norm
cosine_sim_matrix_author = tf.matmul(normalized_matrix,normalized_matrix,transpose_b = True)


In [ ]:
cosine_sim_matrix_author.numpy()

In [ ]:
filled_author_cos = np.nan_to_num(cosine_sim_matrix_author, nan=-1)

In [ ]:
filled_author_cos

In [ ]:
books

In [ ]:
def recommend_author(author_name):
  index = np.where(pt1.index == author_name)[0][0]
  similar_items = sorted(list(enumerate(filled_author_cos[index])),key = lambda x:x[1],reverse = True)[0:50]
  author_name = []
  for i in similar_items:

    author_name.append(pt1.index[i[0]])
    #print(np.where(final_ratings['Book-Author']==pt1.index[i[0]]))

    #print(pt1.index[i[0]])
  #print(author_name)
  index1 = []
  for x in author_name:
    value_to_find = x
    index = books.index[books['Book-Author'].values == value_to_find].tolist()[0]
    index1.append(index)
  data = []
  for i in index1:
    #books = final_ratings.loc[i,'Book-Title']
    #print(books)
    item = []
    temp_df = books[books['Book-Title']==books.loc[i,'Book-Title']]
    item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
    item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
    item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))

    data.append(item)
  return data
  #print(index1)

    #for author_name in final_ratings['Book-Author']:
     #print(final_ratings['Book-Title'])


In [ ]:
final_books

In [ ]:
np.where(pt1.index == 'ANN PACKER')

In [ ]:
pt1

In [ ]:
recommend_author('Yann Queffelec')

In [ ]:
popularity_df['Image-URL-M'][0]

In [ ]:
#import pickle

In [ ]:
#pickle.dump(pt,open('pt.pkl','wb'))
#pickle.dump(books,open('books.pkl','wb'))
#pickle.dump(cosine_sim_matrix,open('cosine_sim_matrix.pkl','wb'))
